In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/KAGGLE_API_CREDENTIALS/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d plameneduardo/a-covid-multiclass-dataset-of-ct-scans

In [ ]:
!unzip /content/a-covid-multiclass-dataset-of-ct-scans.zip

In [ ]:
import torch
from torch import nn
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

In [ ]:
# Define a transform to convert the images to tensors
transform = transforms.Compose([transforms.Resize((224,224)),  # Resize all images to 224x224
                                transforms.ToTensor()])  # Convert the images to tensors

# Load the images from the 'root' directory
dataset = datasets.ImageFolder('New_Data_CoV2', transform=transform)


In [ ]:
test_split = 0.33
shuffle_dataset = True
random_seed=42
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))

In [ ]:
if shuffle_dataset:
  np.random.seed(42)
  np.random.shuffle(indices)

train_indices, test_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

BATCH_SIZE = 32
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)



In [ ]:
from helper_functions import accuracy_fn
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class CovNetScan(nn.Module):
  def __init__(self, input_shape,output):
    super().__init__()
    self.layers_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=35),
        nn.BatchNorm1d(35),
        nn.ReLU(),
        nn.Linear(in_features=35, out_features=50),
        nn.BatchNorm1d(50),
        nn.ReLU(),
        nn.Linear(in_features=50, out_features=40),
        nn.Linear(in_features=40, out_features=output),
    )
  
  def forward(self, X):
    return self.layers_stack(X)

In [ ]:
input_shape = 3 * 224 * 224
num_classes = 3
model_0 = CovNetScan(input_shape = input_shape , output = num_classes).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.06)



In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report
import copy
best_report = None
epochs = 100
best_acc = 0.0
best_model_weights = copy.deepcopy(model_0.state_dict())

for epoch in range(epochs):
    model_0.train()
    for X,y in train_loader:
        X,y = X.to(device),y.to(device)
        y_pred = model_0(X)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:

      with torch.inference_mode():
          model_0.eval()
          y_true = []
          y_pred = []
          
          for X_test,y_test in tqdm(test_loader):
              X_test,y_test = X_test.to(device),y_test.to(device)
              y_test_pred = model_0(X_test)
              test_loss = loss_fn(y_test_pred,y_test)
              y_true.extend(y_test.cpu().numpy())
              y_pred.extend(y_test_pred.argmax(dim=1).cpu().numpy())
          
          report = classification_report(y_true=y_true, y_pred=y_pred, target_names=dataset.classes, output_dict=True)
          current_acc = report['accuracy']
          if best_acc < current_acc: 
              best_acc = current_acc
              best_model_weights = copy.deepcopy(model_0.state_dict())
              best_report = report

          

print(best_report)



In [ ]:
model_0.load_state_dict(best_model_weights)
torch.save(model_0.state_dict(),'Covid_CT_Scans')